In [ ]:
!pip install gradio torch torchvision huggingface_hub --quiet

from huggingface_hub import notebook_login
notebook_login()  # aquí pegas tu token de HF cuando lo pida


In [33]:
!ls -lh /content


total 37M
-rw-r--r-- 1 root root  35M Dec 12 03:31 model_best_torchscript.pt
-rw-r--r-- 1 root root 2.0M Dec 12 04:28 model_futbol_ts.pt
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
drwxr-xr-x 3 root root 4.0K Dec 12 04:24 space_app


In [34]:
import os, shutil

SPACE_DIR = "/content/space_app"
os.makedirs(SPACE_DIR, exist_ok=True)

# mover el modelo a la carpeta del Space
shutil.move("/content/model_futbol_ts.pt",
            os.path.join(SPACE_DIR, "model_futbol_ts.pt"))

print("Contenido de space_app:")
!ls -lh "$SPACE_DIR"


Contenido de space_app:
total 3.1M
-rw-r--r-- 1 root root 1.7K Dec 12 03:31 app.py
-rw-r--r-- 1 root root 3.0M Dec 12 04:28 model_futbol_ts.pt
-rw-r--r-- 1 root root   25 Dec 12 03:31 requirements.txt
drwxr-xr-x 2 root root 4.0K Dec 12 04:17 space_app


In [36]:
import os

for path in ["/content/model_futbol_ts.pt",
             "/content/model_best_torchscript.pt",
             "/content/space_app/model_futbol_ts.pt",
             "/content/space_app/model_best_torchscript.pt"]:
    if os.path.exists(path):
        os.remove(path)
        print("Borrado:", path)

print("Listo, TorchScripts viejos eliminados.")


Borrado: /content/model_futbol_ts.pt
Borrado: /content/model_best_torchscript.pt
Borrado: /content/space_app/model_futbol_ts.pt
Listo, TorchScripts viejos eliminados.


In [37]:
!ls -lh /content


total 43M
-rw-r--r-- 1 root root  43M Dec 12 04:37 model_best.pth
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
drwxr-xr-x 3 root root 4.0K Dec 12 04:31 space_app


In [38]:
import torch
import torch.nn as nn
from torchvision import models

# Parámetros de tu modelo
IMG_SIZE = 224              # el que usaste en el entrenamiento
CLASS_NAMES = ["Clean_Tackles", "Fouls"]
num_classes = len(CLASS_NAMES)

device = torch.device("cpu")
print("Device export:", device)
print("Clases:", CLASS_NAMES, " -> num_classes =", num_classes)

# 1. Crear la misma ResNet18
model = models.resnet18(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# 2. Cargar los pesos entrenados desde model_best.pth
state_dict = torch.load("/content/model_best.pth", map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

print("✅ Modelo original reconstruido en Colab")


Device export: cpu
Clases: ['Clean_Tackles', 'Fouls']  -> num_classes = 2
✅ Modelo original reconstruido en Colab


In [39]:
# 3. Exportar a TorchScript
example_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)

ts_model = torch.jit.trace(model, example_input)
ts_model = ts_model.to(device)
ts_model.eval()

TS_PATH = "/content/model_futbol_ts.pt"
ts_model.save(TS_PATH)

print("✅ TorchScript guardado en:", TS_PATH)


✅ TorchScript guardado en: /content/model_futbol_ts.pt


In [40]:

ts_model2 = torch.jit.load(TS_PATH, map_location="cpu")
print("✅ TorchScript se carga bien en Colab:", type(ts_model2))


✅ TorchScript se carga bien en Colab: <class 'torch.jit._script.RecursiveScriptModule'>


In [41]:
import os, shutil

SPACE_DIR = "/content/space_app"
os.makedirs(SPACE_DIR, exist_ok=True)

shutil.copy(TS_PATH, os.path.join(SPACE_DIR, "model_futbol_ts.pt"))

print("Contenido de space_app:")
!ls -lh "$SPACE_DIR"


Contenido de space_app:
total 43M
-rw-r--r-- 1 root root 1.7K Dec 12 03:31 app.py
-rw-r--r-- 1 root root  43M Dec 12 04:38 model_futbol_ts.pt
-rw-r--r-- 1 root root   25 Dec 12 03:31 requirements.txt
drwxr-xr-x 2 root root 4.0K Dec 12 04:17 space_app


In [42]:
app_py = r"""
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import gradio as gr

IMG_SIZE = 224
CLASS_NAMES = ["Clean_Tackles", "Fouls"]

DEVICE = torch.device("cpu")
MODEL_PATH = "model_futbol_ts.pt"

model = torch.jit.load(MODEL_PATH, map_location=DEVICE)
model.eval()

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

def predict(image: Image.Image):
    image = image.convert("RGB")
    x = transform(image).unsqueeze(0)
    with torch.no_grad():
        logits = model(x.to(DEVICE))
        probs = F.softmax(logits, dim=1)[0].cpu().numpy()

    prob_dict = {CLASS_NAMES[i]: float(probs[i]) for i in range(len(CLASS_NAMES))}
    pred_idx = int(probs.argmax())
    pred_label = CLASS_NAMES[pred_idx]
    return pred_label, prob_dict

title = "Clasificación de Entradas en Fútbol: Falta vs Entrada Limpia"
description = "Sube una imagen de un tackle y el modelo dirá si es una entrada limpia (Clean_Tackles) o una falta (Fouls)."

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Imagen de la jugada"),
    outputs=[
        gr.Label(label="Predicción principal"),
        gr.Label(label="Probabilidades por clase")
    ],
    title=title,
    description=description,
)

if __name__ == "__main__":
    demo.launch()
"""

with open(os.path.join(SPACE_DIR, "app.py"), "w") as f:
    f.write(app_py)

print("app.py creado/actualizado.")


app.py creado/actualizado.


In [47]:
import os, shutil

SPACE_DIR = "/content/space_app"
os.makedirs(SPACE_DIR, exist_ok=True)

# Si aún no lo copiaste/moviste:
# shutil.move("/content/model_futbol_ts.pt", os.path.join(SPACE_DIR, "model_futbol_ts.pt"))

print("Contenido de space_app:")
!ls -lh "$SPACE_DIR"


Contenido de space_app:
total 43M
-rw-r--r-- 1 root root 1.5K Dec 12 04:39 app.py
-rw-r--r-- 1 root root  43M Dec 12 04:38 model_futbol_ts.pt
-rw-r--r-- 1 root root   25 Dec 12 03:31 requirements.txt
drwxr-xr-x 2 root root 4.0K Dec 12 04:17 space_app


In [48]:
%cd /content/space_app
!python app.py


/content/space_app
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 3043, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/space_app/app.py", line 53, in <module>
    demo.launch()
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2950, in launch
    self.block_thread()
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 3045, in block_thread
    print("Keyboard interruption in main thread... closing server.")
KeyboardInterrupt


In [49]:
from huggingface_hub import HfApi

api = HfApi()

# Usa el nombre real de tu Space
repo_id = "juanAG1210/pdi-futbol-tackles"  # o el que hayas creado

api.upload_folder(
    folder_path="/content/space_app",
    repo_id=repo_id,
    repo_type="space"
)

print("✅ Carpeta subida al Space:", repo_id)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...model_best_torchscript.pt:  18%|#8        | 8.32MB / 46.1MB            

  ...ce_app/model_futbol_ts.pt:  19%|#8        | 8.38MB / 44.9MB            

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Carpeta subida al Space: juanAG1210/pdi-futbol-tackles
